# KGG220 Surveying 2: Computational Assignment
### Jordan Rattle
#### Last updated: 2022-10-03

Firstly we will import the required packages to complete this assignment:

In [220]:
from  numpy import *
from  matplotlib.pyplot import *
import folium
import pandas as pd



Now we will add the functions required for computations:

In [221]:
def cot(x):
    cot = round(1/tan(x), 7)
    return cot

#### Function doesn't handle negative degrees, string format "D-M-S"
def dms_radians(dms_string):
    degrees = sum(fromstring(dms_string, sep = '-') * [1.0, 1/60.0, 1/3600.0])
    return radians(degrees)

### This function can handle negative degrees
### but string format has to be "D,M,S"
def dms_radians_neg(dms_string_commasep):
    degrees = sum(fromstring(dms_string_commasep, sep = ',') * [1.0, 1/60.0, 1/3600.0])
    return radians(degrees)

def rad_to_dms(rad):
    'Adaptation of Luke Wallace''s function'
    dd = rad2deg(rad)
    d = floor(dd)
    m = floor((dd-d)*60)
    s = (dd - d - m/60)*3600
    dms = tuple([d,m,s])
    return dms

def rad_to_dmsstring(rad):
    dd = rad2deg(rad)
    d = int(floor(dd))
    m = int(floor((dd - d) * 60))
    s = round((dd - d - m/60)*3600, 3)
    dms = f'{d}-{m}-{s}'
    return dms

# 3 point resection with two internal angles
def resec_tan(EA, NA, EB, NB, EC, NC, angAB, angBC):
    angAB = dms_radians(angAB)
    angBC = dms_radians(angBC)
    num = (EA - EB)*cot(angAB) + (EC - EB)*cot(angBC) - (NC - NA)
    den = (NA - NB)*cot(angAB) + (NC - NB)*cot(angBC) + (EC - EA)
    tanBrgPB = num/den
    brgPB = arctan(tanBrgPB)
    brgPB = brgPB*180/pi
    return brgPB

def Nx_from_AB(EA, NA, Bxa, EB, NB, Bxb):
    'Equation 6 from Bruce Harveys Textbook (Tangent method section)'
    num = (EA - EB) + NB*tan(Bxb) - NA*tan(Bxa)
    den = tan(Bxb) - tan(Bxa)
    Nx = num/den
    return Nx

def Ex_from_BrgXA(EA, NA, NX, brgXA):
    'Equation 3 from Bruce Harveys Textbook (Tangent method section)'
    Ex = EA - (NA - NX)*tan(brgXA) 
    return Ex

def Join(E1, N1, E2, N2):
    dE = E2 - E1 
    dN = N2 - N1
    if dE > 0 and dN < 0 or dE < 0 and dN < 0:
        adder = pi
    elif dE < 0 and dN > 0:
        adder = 2*pi
    elif dE > 0 and dN > 0:
        adder = 0
    D = round(sqrt(dE**2 + dN**2),2)
    B = arctan(dE/dN) + adder
    
    return D, B

def sine_rule_length(BC, angC, angA):
    num = BC*sin(angC) 
    den = sin(angA)
    AB = round(num/den, 2)
    return AB

def Radiation(d, B12, E1, N1):
    dE = d*sin(B12)
    dN = d*cos(B12)
    E2 = round(dE + E1, 2)
    N2 = round(dN + N1, 2)
    return E2, N2

def coord_checker(E1, N1, E2, N2, threshold):
    if abs(E1 - E2) <= threshold:
        print("Eastings values are equivalent!")
    else:
        print("Eastings values diverge!")
    if abs(N1 - N2) <= threshold:
        print("Northings values are equivalent!")
    else:
        print("Northings values diverge!")

def ecc_sqrd(f):
    e2 = 2*f - f**2
    return e2

def rad_curve_mplane(a, e2, geodlat): 
    num = a*(1 - e2)
    den = (1 - e2*(sin(geodlat)**2)**(3/2))
    rcmp = num/den
    return rcmp

def rad_curve_vplane(a, e2, geodlat):
    num = a
    den = sqrt((1 - e2*(sin(geodlat)**2)))
    rcvp = num/den
    return rcvp

def geod2cart(a, e2, geodlat, geodlong, h):
    den = 1 - (e2 * (sin(geodlat)**2))
    den = sqrt(den)
    v = a/den
    vh = v + h
    zv = (1 - e2) * v
    X = vh * cos(geodlat) * cos(geodlong)
    Y = vh * cos(geodlat) * sin(geodlong)
    Z = (zv + h) * sin(geodlat)
    return X, Y, Z

def chord2chord(d2, hA, hB, R):
    num = (d2**2) - (hA - hB)**2
    den = (1 + hA/R) * (1 + hB/R)
    d3 = num/den
    d3 = round(sqrt(d3), 3)
    return d3
    
def chord2arc(d3, R):
    term1 = (d3**2) / (24 * R**2) 
    term2 = (3 * d3**4) / (640 * R**4)
    s = round(d3 * (1 + term1 + term2), 3)
    return s


def getgridcoords(E1, N1, gdBrg, ellipd, False_E, False_N, a, e2):
    c = 111132.952
    c1 = 16038.508
    E1prime = E1  - False_E
    k1 = 0.9996 + (1.23 * (E1prime**2) * 10**-14)
    E2prime = E1prime + (k1 * ellipd * sin(gdBrg))
    N2 = N1 + (k1 * ellipd * cos(gdBrg))
    N2minN1 = N2 - N1
    N1prime = N1 - False_N
    N2prime = N2 - False_N
    NprimeM = (N1prime + N2prime) / 2
    lat1st = (NprimeM/ k1) / c
    lat2nd = ((NprimeM/ k1) + (c1 * sin(2 * lat1st))) / c
    pm = rad_curve_mplane(a, e2, lat2nd)
    vm = rad_curve_vplane(a, e2, lat2nd)
    rm2 = pm * vm * (k1**2)
    num = E1prime**2 + (E1prime * E2prime) + E2prime**2
    term1 = (E1prime**2 + E1prime * E2prime + E2prime**2) / (6 * rm2)
    term2 = 1 + ((E1prime**2 + E1prime * E2prime + E2prime**2) / (36 * rm2))
    K = k1 * (1  + term1 * term2)
    L = ellipd * K
    termone = -N2minN1 * (E2prime + (2 * E1prime))
    termtwo = 1 - (((E2prime + (2 * E1prime))**2) / (27 * rm2))
    sinang1 = (termone * termtwo) / (6 * rm2)
    ang1 = arcsin(sinang1)
    theta = gdBrg + ang1
    terma = N2minN1 * ((2 * E2prime) + E1prime)
    termb = 1 - ((((2 * E2prime) + E1prime)**2) / (27 * rm2))
    sinang2 = (terma * termb) / (6 * rm2)
    ang2 = arcsin(sinang2)
    gdBrg2 = theta + pi - ang2
    dE = L * sin(theta)
    dN = L * cos(theta)
    E2 = round(E1 + dE, 3)
    N2 = round(N1 + dN, 3)
    return E2, N2


### Question 1 - Resection Computation
We have been given the names of the survey marks to perform a three-point resection to find the coordinates of point X: SPM2910, SPM2918 & SPM2777.
After looking on SurCOM (Department of Primary Industries, Parks, Water and Environment, Tasmanian Government 2022)
The coordinates of the survey marks were acquired and put into an excel spreadsheet.
The coordinates will now be put into a dataframe:



In [222]:
control = pd.read_csv("/Users/Jordanius/Desktop/UTAS/KGG220 - Surveying 2/Q1_Control_Marks.csv")
control



,Point,E (m),N (m),Lat,Long
0,SPM2910,528024.890,5256714.276,-42.841633,147.342949
1,SPM2918,528625.198,5256837.276,-42.840503,147.350289
2,SPM2777,528587.779,5256267.776,-42.845633,147.349860


We have also been given the bearings from X to the three survey marks:



In [223]:
control["Obs directions"] = ["0-00-00", "73-54-24.4", "145-54-31.9"]
control



,Point,E (m),N (m),Lat,Long,Obs directions
0,SPM2910,528024.890,5256714.276,-42.841633,147.342949,0-00-00
1,SPM2918,528625.198,5256837.276,-42.840503,147.350289,73-54-24.4
2,SPM2777,528587.779,5256267.776,-42.845633,147.349860,145-54-31.9


Let's map the survey marks to see where they are and the potential location of point X:

In [224]:
marklist = [[control.iloc[i][3], control.iloc[i][4]] for i in range(len(control))]

m = folium.Map(location=[-42.841633344, 147.342948837], zoom_start = 15)
for i in range(len(marklist)):
    m.add_child(folium.Marker(location = marklist[i], icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'star')))
m

It seems that point X will be somewhere in the reserve in Lindisfarne.

Now we will find the two internal angles required to use the Tangent Method:

Let SPM2910 = A, SPM2918 = B & SPM2777 = C



In [225]:
# First we will find the internal angles from the given bearings
a1 = dms_radians(control.loc[1, "Obs directions"])
a2 = rad_to_dmsstring(dms_radians(control.loc[2, "Obs directions"]) - a1)
a1 = rad_to_dmsstring(a1)

print(f'Internal angle 1 = {a1} \nInternal angle 2 = {a2}')



Internal angle 1 = 73-54-24.4 
Internal angle 2 = 72-0-7.5


Having acquired all of the necessary variables, we can now compute the bearing from X to B using the Tangent Method.
The function tan_resec() implements this method, which was obtained from Bruce Harvey's 'Survey Computations' 2020.

In [226]:
# Bearing from X to B in decimal degrees
BXB = resec_tan(control.iloc[0][1], control.iloc[0][2], control.iloc[1][1], control.iloc[1][2],control.iloc[2][1], control.iloc[2][2],a1, a2)

# Bearing from X to B in string format
BXBstr = rad_to_dmsstring(resec_tan(control.iloc[0][1], control.iloc[0][2], control.iloc[1][1], control.iloc[1][2],control.iloc[2][1], control.iloc[2][2],a1, a2)*pi/180)
print(f'Bearing from X to B = {BXBstr}')

Bearing from X to B = 37-20-3.357


It is now possible to calculate the bearings from X to A & X to C:



In [227]:
# Calculate bearing from X to A
A1 = dms_radians(a1)
bxb = dms_radians(BXBstr)
brgXArad = (bxb - A1) + 2*pi
BrgXA = rad_to_dmsstring((brgXBrad - A1) + 2*pi)
brgXCrad = dms_radians(Bxc)
BrgXCrad = brgXCrad + brgXArad - 2*pi
BrgXC = rad_to_dmsstring(BrgXCrad)

print(f'Bearing from X to A = {BrgXA}\nBearing from X to C = {BrgXC}')


Bearing from X to A = 323-25-38.957
Bearing from X to C = 109-20-10.857


We will now calculate the Northing and Easting for point X using equations 6 & 3 from Harvey's (2022) Tangent Method section respectively.


In [228]:
# Using Bruce Harvey's eq 6 (Tangent Method)
NX = round(Nx_from_AB(control.iloc[0][1], control.iloc[0][2], brgXArad,control.iloc[1][1], control.iloc[1][2], bxb), 3)

# Using Bruce Harvey's eq 3
EX = round(Ex_from_BrgXA(control.iloc[0][1], control.iloc[0][2], NX, brgXArad), 3)

# Add X coordinates to the control dataframe
control.loc[2] = ['X', EX, NX, '?', '?', 'N/A']
control


,Point,E (m),N (m),Lat,Long,Obs directions
0,SPM2910,528024.890,5256714.276,-42.841633,147.342949,0-00-00
1,SPM2918,528625.198,5256837.276,-42.840503,147.350289,73-54-24.4
2,X,528274.631,5256377.662,?,?,N/A


After using Geoscience Australia's tool 'Grid to Geographic'
We now have the latitude and longitude values for point X.


In [229]:
Xlat = -42.844655417
Xlong = 147.346021853

#Add the Geographic coordinates to a new control dataframe
Xcontrol = control.copy()
Xcontrol.loc[2, "Lat"] = Xlat
Xcontrol.loc[2, "Long"] = Xlong
Xcontrol



,Point,E (m),N (m),Lat,Long,Obs directions
0,SPM2910,528024.890,5256714.276,-42.841633,147.342949,0-00-00
1,SPM2918,528625.198,5256837.276,-42.840503,147.350289,73-54-24.4
2,X,528274.631,5256377.662,-42.844655,147.346022,N/A


We will now map our computed geographic coordinates for point X:

In [230]:
m.add_child(folium.Marker(location = [-42.844655417,147.346021853], icon = folium.Icon(color = 'pink', prefix = 'fa', icon = 'star')))
m

The pink marker indicates the location of point X, which seems to be at top of Natone Hill.

# Question 2 - Intersection
We are required to find the intersection of the lines QP and RP i.e. the coordinates of P.
The coordinates and internal angles of Q & R have been provided.

First, we will put our given data into a dataframe:

In [231]:
intanglist = ['23-12-43', '42-52-43', '?']

for i in range(len(intanglist) - 1):
    intanglist[i] = dms_radians(intanglist[i])
    
q2coords = pd.DataFrame({'Point': ['Q', 'R', 'P'], 'E (m)': [2743.19, 2642.84, '?'], 'N (m)': [5122.54, 4625.21, '?'], 'Internal ang': intanglist})
q2coords

,Point,E (m),N (m),Internal ang
0,Q,2743.19,5122.54,0.405125
1,R,2642.84,4625.21,0.748373
2,P,?,?,?


We start by finding the third internal angle of the triangle QPR:   

In [232]:
# Compute internal angle of P & add it to dataframe
angP = pi - (q2coords.loc[0, "Internal ang"] + q2coords.loc[1, "Internal ang"])
q2coords.loc[2, "Internal ang"] = angP
q2coords



,Point,E (m),N (m),Internal ang
0,Q,2743.19,5122.54,0.405125
1,R,2642.84,4625.21,0.748373
2,P,?,?,1.988095


The code below will walk through the process of finding the coordinates of P and returns the output:

In [233]:
# Compute Join of Q & R to get bearing Q to R and distance QR
join = Join(q2coords.loc[0, "E (m)"], q2coords.loc[0, "N (m)"], q2coords.loc[1, "E (m)"], q2coords.loc[1, "N (m)"])
dQR = join[0]
BrgQR = join[1]

# Compute bearings from R to P & P to Q
BrgRP = BrgQR - q2coords.loc[1, "Internal ang"] + pi
BrgPQ = BrgRP - pi - q2coords.loc[2, "Internal ang"]

# Now we will compute the length of the line PQ using the sine rule
dPQ = sine_rule_length(dQR, q2coords.loc[1, "Internal ang"], q2coords.loc[2, "Internal ang"])

# Now we have everything we need to compute the coordinates of P
# We will now compute a Radiation from Q to P to get the coordinates of P
BrgQP = BrgPQ + pi
Pcoords = Radiation(dPQ, BrgQP, q2coords.loc[0, "E (m)"], q2coords.loc[0, "N (m)"])
print(f'Easting (m) of P = {Pcoords[0]}\nNorthing (m) of P = {Pcoords[1]}')



Easting (m) of P = 2528.65
Northing (m) of P = 4811.77


In [234]:
# We will now do a check to see if the results are consistent using the other leg of the triangle 
# with a threshold of the difference defined in the function as |difference| <= 0.005m. 
# Any two values that have diffferences <= 5mm will be considered an approximate equality.

dRP = sine_rule_length(dQR, q2coords.loc[0, "Internal ang"], q2coords.loc[2, "Internal ang"])
Pcoords1 = Radiation(dRP, BrgRP, q2coords.loc[1, "E (m)"], q2coords.loc[1, "N (m)"])
threshold = 0.005
coord_checker(Pcoords[0], Pcoords[1], Pcoords1[0], Pcoords1[1], threshold)



Eastings values are equivalent!
Northings values are equivalent!


In [235]:
# Since the computed coordinates passed the check, we will add the
# P coordinates to the q2 dataframe
q2coords.loc[2, "E (m)"] = Pcoords[0]
q2coords.loc[2, "N (m)"] = Pcoords[1]

q2coords



,Point,E (m),N (m),Internal ang
0,Q,2743.19,5122.54,0.405125
1,R,2642.84,4625.21,0.748373
2,P,2528.65,4811.77,1.988095


## Question 3 - Datum states
### Static Datum vs Dynamic Datum
A static datum refers to a datum that has been set to one point in space time (epoch) eg GDA2020 was set on 2020-01-01. A dynamic datum is fluid in that it takes into account continual drift of the earth's surface and changes based on changing parameters.

### Global Ellipsoid vs Local Ellipsoid
A global ellipsoid is an approximation that best fits the shape of the earth globally eg WGS84.
A local ellipsoid is an approximation of the shape of the earth that best fits the earth in the local area eg GDA2020.

## Question 4 - Different Datums

Part 1
AGD66 was defined using the Geodetic Reference System ellipsoid with the
flattening term accurate to two decimal places. The ellipsoid was renamed
the Australian National Spheroid (ANS).
AGD66's direction of minor axis was defined as being parallel to 
the Conventional International Origin (CIO).
The reference meridian plane of zero longitude was defined as being parallel
to Bureau International de l'Heure (BIH) mean meridian plane.
The positional centre of the ANS was defined as the coordinates for the
Johnstone Geodetic Station.

GDA94 was defined using the GRS80 ellipsoid. GRS80 has its geometric centre 
at the centre of the mass of the earth whereas ANS's geomtric centre
diverges by about 200m.
The coordinates used to define GDA94 were the GNSS (at the time GPS) 
observations from the Australian National Network (ANN). Coordinates were 
determined using these observations with reference to the International
Terrestrial Reference Frame 1992 (ITRF92) with a common epich of 1994.0.
GDA94 is aligned to global reference frame while AGD66 is not.

Part 2
GDA2020 coordinates were determined by a least squares adjustment on a 3D
network of all available data (including GNSS) from related stakeholders. 
GDA2020 is based on the ITRF2014 with an epoch at 2020.0. 
GDA2020 is constrained using the Asia-Pacific Reference Frame (APREF) time
series combination solution. This solution is computed weekly for APREF
within Australian jursidiction and thus provides a link between the ITRF2014
and GDA2020.

WGS84 is based on the ITRF2014. WGS84 is adjusted annually to account for
plate tectonic motion.



## Question 5 - Computing Cartesian Coordinates

In [236]:
# Part 1 
## Source:  GDA2020 Manual, ICSM 
# GRS80 semi-major axis in (m)
a_GRS80 = 6378137.0
# GRS80 inverse flattening
f_GRS80 = 1/298.257222101
f1_GRS80 = f_GRS80 - 1

## Source: GDA2020 Manual, ICSM 
# ANS semi-major axis in (m)
a_ANS = 6378160.0
# ANS inverse flattening
f_ANS = 1/298.25
f1_ANS = f_ANS - 1

## Source: https://earth-info.nga.mil/index.php?dir=wgs84&action=wgs84
# WGS84 semi-major axis in (m)
a_WGS84 = 6378137.0
# WGS84 inverse flattening
f_WGS84 = 1/298.257223563
f1_WGS84 = f_WGS84 - 1

# Part 2
# Compute first eccentricity^2 for GRS80, ANS, WGS84
e2_GRS80 = ecc_sqrd(f_GRS80)
e2_ANS = ecc_sqrd(f_ANS)
e2_WGS84 = ecc_sqrd(f_WGS84)


# Part 3
# Geodetic Latitude
geod_lat = '-38,-11,-49'
# Geodetic Longitude
geod_long = "144-59-02"
# height in (m)
h = 48

# Compute prime meridian radius of curvature
glatrad = dms_radians_neg(geod_lat)
glongrad = dms_radians(geod_long)

rcmp = rad_curve_mplane(a_GRS80, e2_GRS80, glatrad)

# Compute prime vertical radius of curvature
rcvp = rad_curve_vplane(a_GRS80, e2_GRS80, glatrad)

# Compute mean radius of curvature
RCmean = sqrt(rcmp*rcvp)

# Compute equivalent XYZ coordinates for GRS80
XYZ_GRS80 = geod2cart(a_GRS80, e2_GRS80, glatrad, glongrad, h)

# Compute prime vertical & prime meridian radii of curvature for ANS 
ANS_rcvp = rad_curve_vplane(a_ANS, e2_ANS, glatrad)
ANS_rcmp = rad_curve_mplane(a_ANS, e2_ANS, glatrad)
ANS_RCmean = sqrt(ANS_rcmp * ANS_rcvp)

# Compute equivalent XYZ coordinates for ANS
XYZ_ANS = geod2cart(a_ANS, e2_ANS, glatrad, glongrad, h)

xyz = pd.DataFrame({'Datum': ['GRS80', 'ANS'], 'X (m)': [XYZ_GRS80[0], XYZ_ANS[0]], 'Y (m)': [XYZ_GRS80[1], XYZ_ANS[1]], 'Z (m)': [XYZ_GRS80[2], XYZ_ANS[2]]})
# print(f'GRS80: {XYZ_GRS80}\nANS: {XYZ_ANS}\nAbsolute difference: {XYZ_absdiffr}')
xyz


,Datum,X (m),Y (m),Z (m)
0,GRS80,-4.110497e+06,2.879924e+06,-3.922677e+06
1,ANS,-4.110512e+06,2.879934e+06,-3.922690e+06


#### Part 4
Now we will compute the absolute difference between the cartesian coordinates using datums GRS80 and ANS:

In [237]:
# Compute absolute difference when using different ellipisoids
XYZ_absdiff = [abs(XYZ_GRS80[i] - XYZ_ANS[i]) for i in range(len(XYZ_GRS80))]

# Given that h = 48, we must round our answers to this level of precision
XYZ_absdiffr = [round(abs(XYZ_GRS80[i] - XYZ_ANS[i])) for i in range(len(XYZ_GRS80))]

diff = pd.DataFrame([[XYZ_absdiffr[0],XYZ_absdiffr[1],XYZ_absdiffr[2]]], columns = ['dX (m)','dY (m)','dZ (m)'])
diff



,dX (m),dY (m),dZ (m)
0,15,10,14


#### Part 5
The implications of providing incorrect datum information would
most likely result in a displacement of the
position of the spatial data. Using the example above, when using different
datums but with the same geographical coordinates,
the Cartesian coordinates differed by up to 15m between 
the different datums selected. This is obviously more pronounced between the GRS80 ellipsoid
and the ANS. WGS84 and GRS80 are more closely aligned but there is a significant difference
when precise postion is required.


In [238]:
# The location of the point for this question.
mappy = folium.Map(location= [-38.19694444444444,144.98388888888888], zoom_start = 12)
mappy.add_child(folium.Marker(location = [-38.19694444444444,144.98388888888888], icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'star')))
mappy

## Question 6 - Ellipsoidal Distance Computation

In [239]:
# Distance between points A & B (in metres)
dAB = 21154.842
# Heights of A & B (in metres)
hA = 37
hB = 1780
# Approximate radius of the earth (in metres)
R = 6378000

# Given these parameters we will compute the ellipsoidal distance (sAB)
d3 = chord2chord(dAB, hA, hB, R)

# Compute ellipsoidal distance from A to B
s = chord2arc(d3, R)

archord = pd.DataFrame([[d3, s]], columns = ['Chord dist (m)', 'Arc dist (m)'])
archord



,Chord dist (m),Arc dist (m)
0,21079.912,21079.922


## Question 7 - Compute Grid Coordinates (MGA2020)

Note: Diagrams have been provided in seperate files as they are too large to be uploaded to GitHub. 

In [240]:
# Compute the Coordinates for point B using
# E & N for point A, the grid bearing from A to B
# and the ellipsoidal distance from A to B
EA = 568655.583
NA  = 5374392.798
gdBrg = dms_radians("52-41-38")
sAB = 10739.061
False_E = 5*10**5
False_N = 10**7

# Compute coordinates for point B
ptBcoords = getgridcoords(EA, NA, gdBrg, sAB, False_E, False_N, a_GRS80, e2_GRS80)
ptBcoords



(577195.128, 5380899.705)

Now let's map the locations of A and B, A is the red marker and B is the blue marker.

In [241]:
n = folium.Map(location=[-41.779343047, 147.826146023], zoom_start = 12)

# Point A
n.add_child(folium.Marker(location = [-41.779343047,147.826146023], icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'star')))

# Point B
n.add_child(folium.Marker(location = [-41.719957811, 147.928047291], icon = folium.Icon(color = 'blue', prefix = 'fa', icon = 'star')))
n


## Question 8 - GDA94 to MGA94 

#### Part 1

Used the excel spreadsheets to convert coordinates from GDA94 to MGA94

Easting of point = 329,976.918 m

Northing of point = 6,431,941.924 m

Grid convergence: -0°57'46.084"

#### Part 2

Change the zone from 56 to 58
Easting becomes: -805,984.057 m
Northing becomes: 6,348,438.665 m 
Grid convergence: -7°28'05.966"



In [242]:
map8 = folium.Map(location= [-32.236312058,145.195448947], zoom_start = 3)

# MGA94 Zone 56
map8.add_child(folium.Marker(location = [-32.236312058,145.195448947], icon = folium.Icon(color = 'green', prefix = 'fa', icon = 'star')))

# MGA94 Zone 58
map8.add_child(folium.Marker(location = [-32.23631206, 165], icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'star')))

map8

Map shows the location of the GDA94 Zone 56 coordinates (green marker) & the GDA94 Zone 58 coordinates (red marker)

#### Part 3

The differences between the coordinates when choosing 
zones 56 & 58 are seen in both the Eastings and Northings. However,as can be seen in the map above, the
difference in Easting between the two results is of a much higher magnitude.
This is due to the zone change as the zones are meridinal zones and so 
Eastings are innately affected to incorrect zonal definitions. 
The scale factor will be affected along with the redefining of the false origin,
which will result in an incorrect shift of coordinates.


## Question 9 - AMG66 to MGA94
#### Part 1
AMG66 to AGD66 coords (using GA online tool)
34° 5' 32.78073" S, 150° 57' 50.87857" E
2nd attempt 
34° 5' 32.78073" S, 150° 57' 50.87857" E

#### Part 2
AGD66 to Cartesian coordinates
X: -4,667,063.134
Y: 2,485,270.281
Z: -3,554,954.920

2nd attempt
X: -4,622,978.370 
Y:  2,566,343.245
Z: -,3,554,954.920

#### Part 3
7 parameter transformation
X: -4667190.094
Y: 2485217.952
Z: -3554802.363

2nd attempt
X:	-4623105.436
Y:	2566290.942
Z:	-3554802.339


#### Part 4
Cartesian GDA94 to Geodetic GDA94
Latitude: -34°05'27.07141"
Longitude: 151°57'55.00817"
Ellipsoidal Height: 9.738

2nd attempt
Cartesian GDA94 to Geodetic GDA94
Latitude: -34°05'27.10271"
Longitude: 150°	57'	55.06818"
Ellipsoidal Height: 8.270


#### Part 5
Convert GDA94 to MGA94
Easting: 404545.761
Northing: 6227287.187

2nd attempt
Convert GDA94 to MGA94
Easting: 312285.078
Northing: 6225900.674



In [243]:
map9 = folium.Map(location= [-34.092439093, 150.964132937], zoom_start = 16)

# AMG66 Zone 56
map9.add_child(folium.Marker(location = [-34.092439093, 150.964132937], icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'star')))
# MGA94 Zone 56 2nd attempt
map9.add_child(folium.Marker(location = [-34.090861860, 150.965296716], icon = folium.Icon(color = 'blue', prefix = 'fa', icon = 'star')))


map9

## Question 10 - MGA94 to MGA2020

#### Part 1
Convert MGA94 to MGA2020

MGA2020 Easting: 312285.552
MGA2020 Northing: 6225902.093


Convert GDA94 to GDA2020

GDA2020 latitude: 34° 5' 27.15061"
GDA2020 longitude: 150° 57' 55.11282"   
 

In [244]:
# Compute difference (in metres) between MGA94 & MGA2020 coordinates
E_MGA94 = 312285.078
N_MGA94 = 6225900.674
E_MGA2020 = 312285.552
N_MGA2020 = 6225902.093
E_diff = round(E_MGA2020 - E_MGA94, 3)
N_diff =round(N_MGA2020 - N_MGA94, 3)

print(f'The difference between the Eastings of MGA94 & MGA2020 coordinates was {E_diff}m')
print(f'The difference between the Northings of MGA94 & MGA2020 coordinates was {N_diff}m')



The difference between the Eastings of MGA94 & MGA2020 coordinates was 0.474m
The difference between the Northings of MGA94 & MGA2020 coordinates was 1.419m


#### Part 2

The coordinates are not the same due to the fact that MGA2020 has 
been adjusted for the apparent continental drift that has taken place 
and, therefore, has improved postional accuracy when compared with MGA94.
MGA94 is was defined at the 1994.0 epoch whereas MGA2020 was defined at
the 2020.0 epoch.

We will now plot the GDA94 & GDA2020 coordinates on a map: 

In [245]:
map10 = folium.Map(location= [-34.090875169, 150.965309117], zoom_start = 25)

# MGA2020
map10.add_child(folium.Marker(location = [-34.090875169, 150.965309117], icon = folium.Icon(color = 'pink', prefix = 'fa', icon = 'star')))

# MGA94
map10.add_child(folium.Marker(location = [-34.090853169, 145.965280051], icon = folium.Icon(color = 'red', prefix = 'fa', icon = 'star')))

map10


As can be seen, the map does not zoom to the scale required to see the differences between the two.
However, it is important to note that the differences exist.